# Machine learning for public policy
## HW3; Improving pipeline

### Takuya Ando

In this part, I used improved pipeline and experiment several classifiers with different parameters.

In [26]:
# Import packages
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as accuracy
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, average_precision_score, roc_curve, roc_auc_score

### 1. Prepare data for test using

In [8]:
df = pd.read_csv("projects_2012_2013.csv") # Read data
df = df.dropna(how="any")

# Make dummy variables
col_to_transform = ["school_metro", "primary_focus_area", "secondary_focus_area", "resource_type", "poverty_level", "grade_level", "eligible_double_your_impact_match"]
df = pd.get_dummies(df, dummy_na=False, columns = col_to_transform, drop_first=True)

# Make variable for whether being funded within 60days 
date = lambda x: datetime.datetime.strptime(x, '%m/%d/%y')
df["date_posted"] = df["date_posted"].map(date)
df["datefullyfunded"] = df["datefullyfunded"].map(date)

after_60days = datetime.timedelta(days=60)
df["not_within_60days"] = df["datefullyfunded"] > df["date_posted"] + after_60days
df["not_within_60days"] = df["not_within_60days"].astype(int)

In [9]:
def time_split(df, span, start_time, col_name, x_cols, y_col): # split function for time series data
    train_start = datetime.datetime.strptime(start_time, '%m/%d/%Y')
    train_end = train_start + relativedelta(months=span)
    test_start = train_end + relativedelta(days=1)
    test_end = test_start + relativedelta(months=span)
    filter_train1 = train_start <= df[col_name]
    filter_train2 = df[col_name] <= train_end
    filter_test1 = test_start <= df[col_name]
    filter_test2 = df[col_name] <= test_end
    train_data = df[filter_train1][filter_train2]
    train_x = train_data[x_cols]
    train_y = train_data[y_col]
    test_data = df[filter_test1][filter_test2]
    test_x = test_data[x_cols]
    test_y = test_data[y_col]
    return train_x, test_x, train_y, test_y

In [30]:
# Split data into train data and test data
start_date = "01/01/2012"

cols_to_use = ['school_metro_suburban',
       'school_metro_urban','poverty_level_highest poverty', 'poverty_level_low poverty',
       'poverty_level_moderate poverty','students_reached',
       'eligible_double_your_impact_match_t']

train_x, test_x, train_y, test_y = time_split(df, 6, start_date,"date_posted", cols_to_use, 'not_within_60days')

### 2. Prepare classifiers and evaluaters

In [31]:
# Define clasifiers
def k_neighbors(x_train, y_train, k):
    knn = KNeighborsClassifier(n_neighbors=k, metric='minkowski')
    return knn.fit(x_train, y_train)

def decision_tree(x_train, y_train, depth):
    dec_tree = DecisionTreeClassifier(random_state=0, max_depth=depth) 
    return dec_tree.fit(x_train, y_train)

def lsvm(x_train, y_train, penalty, c):
    lsvm = LinearSVC(random_state=0, penalty=penalty, C=c)
    return lsvm.fit(x_train, y_train)

def logistic_regression(x_train, y_train, penalty, c):
    log_model = LogisticRegression(random_state=0, penalty=penalty, C=c) 
    return log_model.fit(x_train, y_train)

def random_forest(x_train, y_train, n):
    r_for = RandomForestClassifier(n_estimators=n, random_state=0)
    return r_for.fit(x_train, y_train)

In [14]:
# Define evaluater
def evaluate_model(model, x_test, y_test, threshold):
    if str(type(model)) == "<class 'sklearn.svm.classes.LinearSVC'>":
        prob_pos = model.decision_function(x_test)
        prob_pos = (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())
        pred_prob = [1-x for x in prob_pos]
        pred_label = [1 if (1-x) >threshold else 0 for x in pred_prob]
    else:
        pred_scores = model.predict_proba(x_test)
        pred_prob = pred_scores[:, 1]
        pred_label = [1 if x[1]>threshold else 0 for x in pred_scores]
    accuracy = accuracy_score(y_test, pred_label)
    precision = precision_score(y_test, pred_label)
    recall = recall_score(y_test, pred_label)
    f1 = f1_score(y_test, pred_label)
    ap = average_precision_score(y_test, pred_prob)
    auc = roc_auc_score(y_test, pred_prob)
    return accuracy, precision, recall, f1, ap, auc

### 3. Experiment with different parameters

In [17]:
for num_neighbors in [1,3,10,20,50]:
    model = k_neighbors(train_x, train_y, num_neighbors)
    metrics = evaluate_model(model, test_x, test_y, 0.3)
    print("KNN; k=" + str(num_neighbors))
    print("accuracy, precision, recall, f1, ap, auc")
    print(metrics)

KNN; k=1
accuracy, precision, recall, f1, ap, auc
(0.5931537908532876, 0.2646349862258953, 0.33246809431105345, 0.2946984948710574, 0.25864021319954406, 0.507578772266968)
KNN; k=3
accuracy, precision, recall, f1, ap, auc
(0.43648730852181605, 0.25873277281789026, 0.645684620376379, 0.3694306930693069, 0.2604067654474682, 0.509221705928655)
KNN; k=10
accuracy, precision, recall, f1, ap, auc
(0.5484709395564895, 0.26654363300817296, 0.4373783257624919, 0.33123105905479566, 0.2644866843007689, 0.518225945021324)
KNN; k=20
accuracy, precision, recall, f1, ap, auc
(0.519880550793563, 0.2667509481668774, 0.5020549426779147, 0.34839387571299907, 0.2710255059502071, 0.5247799779447616)
KNN; k=50
accuracy, precision, recall, f1, ap, auc
(0.5229220815130233, 0.2754094682521876, 0.5310404499242916, 0.3627096106966093, 0.2749610159743521, 0.5361318689195022)


In [21]:
for depth in [1, 3, 5, 10, 20]:
    model = decision_tree(train_x, train_y, depth)
    metrics = evaluate_model(model, test_x, test_y, 0.3)
    print("DecisionTree; depth=" + str(depth))
    print("accuracy, precision, recall, f1, ap, auc")
    print(metrics)

DecisionTree; depth=1
accuracy, precision, recall, f1, ap, auc
(0.46037714980921307, 0.2873001408334024, 0.7501622323166774, 0.4154786150712831, 0.27939386010114653, 0.5555045931271352)
DecisionTree; depth=3
accuracy, precision, recall, f1, ap, auc
(0.4797323452966875, 0.2879177377892031, 0.7025740860912827, 0.40845070422535207, 0.2847245430871921, 0.5574530763714858)
DecisionTree; depth=5
accuracy, precision, recall, f1, ap, auc
(0.5321572747884754, 0.290807981681387, 0.5768981181051266, 0.3866898651587647, 0.28744302599191224, 0.5585079480175192)
DecisionTree; depth=10
accuracy, precision, recall, f1, ap, auc
(0.49870043687441246, 0.2814406612871482, 0.6186459009301319, 0.3868785931687521, 0.2831600002680108, 0.5405938602741831)
DecisionTree; depth=20
accuracy, precision, recall, f1, ap, auc
(0.5061107117181883, 0.27435575109993715, 0.5665152498377677, 0.3696802879525725, 0.27687308822945783, 0.527635033695146)


In [33]:
for c in [0.1, 1, 10, 100]:
    model = lsvm(train_x, train_y, "l2", c)
    metrics = evaluate_model(model, test_x, test_y, 0.3)
    print("SVM; C=" + str(c))
    print("accuracy, precision, recall, f1, ap, auc")
    print(metrics)

SVM; C=0.1
accuracy, precision, recall, f1, ap, auc
(0.5875684344411879, 0.2994198386868544, 0.4577114427860697, 0.36201881950384945, 0.21950616141338442, 0.4312993064909961)
SVM; C=1
accuracy, precision, recall, f1, ap, auc
(0.26024442846872753, 0.255529490616622, 0.9896171317326411, 0.40617925156478896, 0.24690684787663975, 0.46534038734552574)
SVM; C=10
accuracy, precision, recall, f1, ap, auc
(0.25985732455897803, 0.2555115253669699, 0.9902660609993511, 0.4062111801242236, 0.2562568200679136, 0.48843134125869136)
SVM; C=100
accuracy, precision, recall, f1, ap, auc
(0.2599126251175137, 0.2555530751199911, 0.9904823707549211, 0.4062818863404463, 0.25794326580105936, 0.4945279497595684)


In [32]:
for c in [0.1, 1, 10, 100]:
    model = logistic_regression(train_x, train_y, "l2", c)
    metrics = evaluate_model(model, test_x, test_y, 0.3)
    print("Log; C=" + str(c))
    print("accuracy, precision, recall, f1, ap, auc")
    print(metrics)

Log; C=0.1
accuracy, precision, recall, f1, ap, auc
(0.46850633191395236, 0.28824927831550345, 0.7343716201600692, 0.41399914639351254, 0.3062994029300311, 0.5750224505741851)
Log; C=1
accuracy, precision, recall, f1, ap, auc
(0.4700547475529503, 0.2885763000852515, 0.7322085226043694, 0.4139913165779979, 0.3062254021435348, 0.5748323840452753)
Log; C=10
accuracy, precision, recall, f1, ap, auc
(0.4702206492285572, 0.28865012364628634, 0.7322085226043694, 0.4140672782874618, 0.3062243305298483, 0.574827530735752)
Log; C=100
accuracy, precision, recall, f1, ap, auc
(0.4702206492285572, 0.28865012364628634, 0.7322085226043694, 0.4140672782874618, 0.3062236967184856, 0.5748268236310533)


In [29]:
for n in [1, 5, 10, 20, 50, 100]:
    model = random_forest(train_x, train_y, n)
    metrics = evaluate_model(model, test_x, test_y, 0.3)
    print("RF; n_estimator=" + str(n))
    print("accuracy, precision, recall, f1, ap, auc")
    print(metrics)

RF; n_estimator=1
accuracy, precision, recall, f1, ap, auc
(0.5202676547033125, 0.26982503976369004, 0.5137356694786935, 0.3538175046554935, 0.26943727025098463, 0.519899163655847)
RF; n_estimator=5
accuracy, precision, recall, f1, ap, auc
(0.5226455787203451, 0.2738350445673023, 0.5249837767683323, 0.3599288150674774, 0.2747364066933434, 0.5282218823834185)
RF; n_estimator=10
accuracy, precision, recall, f1, ap, auc
(0.5186639385057789, 0.27249414650462706, 0.5286610426130218, 0.3596233078281342, 0.2763924588475094, 0.5278884985885226)
RF; n_estimator=20
accuracy, precision, recall, f1, ap, auc
(0.5107559586351822, 0.2733419188667096, 0.5509409474367294, 0.3653970303421563, 0.27684411759271255, 0.528910216666522)
RF; n_estimator=50
accuracy, precision, recall, f1, ap, auc
(0.5130232815351435, 0.2726383302532884, 0.5425048669695003, 0.3628997250759658, 0.27627493388818103, 0.5276498668232582)
RF; n_estimator=100
accuracy, precision, recall, f1, ap, auc
(0.500912459215838, 0.27360625385